In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import seaborn as sns
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import tqdm 
import os
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, NMF, SlopeOne, CoClustering    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-ratings/user_ratings.csv
/kaggle/input/games-after-2010/games_after_2010.csv


In [2]:
with open('..data/data_for_model/data_for_model.pkl', 'rb') as f:
    # Load the contents of the file using pickle.load()
    data_for_model = pickle.load(f)

Using the recommended metrics by scikit suprise, we will be using recall@k and precision@k to compute the performance of our model. Precision is essentially how many of our predictions our model gave us that we deemed to be relevant to our user in our top 'k' recommendations. Recall is essentially how many of the predictions our model gave out of all relevant items in our top 'k' recommendations, essentially the variability of our recommendation when looking at all relevant recommendations as a whole.

In our specific use case our threshold would be 7.5 and our k will be set to 10.

Refer to the following [link](https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54) for a more detailed explanation.

In [32]:
algos = [('SVD', SVD(random_state=42)),
    ('NonNegative Matrix Factorization', NMF(random_state=42)),
    ('Slope One', SlopeOne()),
    ('Co-clustering', CoClustering(random_state=42))]

In [35]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(data_for_model[['Username','BGGId', 'Rating']],reader)

In [38]:
from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls



We used a 5 fold cross validation to train and test our model to determine the metrics we want including our loss (RMSE).

In [41]:
# # # # #############really need to sort this out ####################
precs_list = []
recalls_list = []
kf = KFold(n_splits=5,random_state=42)
result_all = []
for var_name,model in algos:
    temp_prec_list = []
    temp_recall_list = []
    for trainset, testset in kf.split(data):
        temp_result= []
        model.fit(trainset)
        predictions = model.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7.5)
        result = cross_validate(model,data,measures=['RMSE','MAE'],cv=5,verbose=True)
        temp_result.append(result['test_rmse'].mean())
        result_all.append((var_name,temp_result))
    # Precision and recall can then be averaged over all users
        temp_prec_list.append(sum(prec for prec in precisions.values()) / len(precisions))
        temp_recall_list.append(sum(rec for rec in recalls.values()) / len(recalls))
    precs_list.append((var_name,np.mean(temp_prec_list)))
    recalls_list.append((var_name,np.mean(temp_recall_list)))
    

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0204  1.0215  1.0225  1.0218  1.0207  1.0214  0.0008  
MAE (testset)     0.7584  0.7592  0.7586  0.7583  0.7579  0.7585  0.0004  
Fit time          48.92   49.68   50.04   49.64   49.96   49.65   0.40    
Test time         1.80    2.29    1.89    1.78    1.79    1.91    0.19    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0217  1.0165  1.0219  1.0216  1.0207  1.0205  0.0020  
MAE (testset)     0.7584  0.7560  0.7588  0.7590  0.7579  0.7580  0.0011  
Fit time          49.00   49.59   49.81   49.38   49.10   49.38   0.30    
Test time         1.91    2.25    1.79    1.79    2.25    2.00    0.21    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0183  1.0179  1.0155  1.0178  1.0177  1.0174  0.0010  
MAE (testset)     0.7587  0.7587  0.7586  0.7593  0.7579  0.7587  0.0004  
Fit time          9.73    9.66    9.68    9.95    9.98    9.80    0.14    
Test time         29.90   29.31   30.04   28.69   29.61   29.51   0.48    
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0181  1.0210  1.0157  1.0142  1.0186  1.0176  0.0024  
MAE (testset)     0.7592  0.7608  0.7569  0.7575  0.7590  0.7587  0.0014  
Fit time          9.52    9.96    9.88    9.76    10.03   9.83    0.18    
Test time         29.29   29.17   29.89   29.37   29.08   29.36   0.28    
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std    

In [42]:
recalls_list

[('SVD', 0.3900891080143896),
 ('NonNegative Matrix Factorization', 0.009557127975592088),
 ('Slope One', 0.3605168296879965),
 ('Co-clustering', 0.33374577517382936)]

In [43]:
precs_list

[('SVD', 0.6981773132017087),
 ('NonNegative Matrix Factorization', 0.08749428313330752),
 ('Slope One', 0.679874590269719),
 ('Co-clustering', 0.6659222609369009)]

In [44]:
result_all

[('SVD', [1.0213712081874704]),
 ('SVD', [1.0204828673113941]),
 ('SVD', [1.0203655120368236]),
 ('SVD', [1.0200507047167389]),
 ('SVD', [1.0200749272648115]),
 ('NonNegative Matrix Factorization', [1.7844759821295415]),
 ('NonNegative Matrix Factorization', [1.777024081528765]),
 ('NonNegative Matrix Factorization', [1.7885889749663555]),
 ('NonNegative Matrix Factorization', [1.7871589997098034]),
 ('NonNegative Matrix Factorization', [1.785430450313568]),
 ('Slope One', [1.0174268302254594]),
 ('Slope One', [1.0175506091427093]),
 ('Slope One', [1.0174059117435126]),
 ('Slope One', [1.0175889050438958]),
 ('Slope One', [1.0175749963301826]),
 ('Co-clustering', [1.045843777282942]),
 ('Co-clustering', [1.0469009270176166]),
 ('Co-clustering', [1.045874629570217]),
 ('Co-clustering', [1.0448459966189174]),
 ('Co-clustering', [1.0448217456155375])]

In [45]:
import pickle
def pickle_lists(list_of_lists):
    for i, sublist in enumerate(list_of_lists):
        with open(f"../data/pickled_advanced/{i}.pickle", "wb") as f:
            pickle.dump(sublist, f)

In [46]:
pickle_lists([recalls_list,precs_list,result_all])

## Conclusion

Due to computational constraints(long run time and memory errors) our modelling and metrics will be done in seperate notebooks and the data was pickled for further examination in model_final notebook.